<div style="background-color: #F0F0F0; padding: 10px; font-size: 32px; font-weight: bold; text-align: center; border-radius: 10px;">
  <span style="color: green;">S423 - Automatisk Oppretninger ved hjelp av Machine Learning</span>
</div>



---

## Table of Contents

1. [Introduction](#Introduction)
2. [Setup and Configuration](#Setup-and-Configuration)
3. [Data Loading and Preprocessing](#Data-Loading-and-Preprocessing)
4. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
5. [Model Building](#Model-Building)
6. [Evaluation](#Evaluation)
7. [Conclusion](#Conclusion)
8. [References](#References)

---


# Import necessary libraries

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import geopandas as gpd
import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

# Data exploration

This step requires the filtering of dataframes so that the correct nærings are included. All bedfrifter will be included in analysis, not just those belonging to reg_type 2. Also, and very importantly, if running for an entire 'delreg' (or an equivalent) then a backup file needs to be saved. 

We will also need to establish what category the foretak that is being automatically corrected belongs to. There will be three cateogories:

- Good quality data. This will require a simple function to correct. It will resemble what occurred with '999'  in the old method. 

- Bad quality data. Here we will need to use K means in order to find nearest neighbors. This can be shown visually for fun. After finding the nearest neighbors can do one of a few things which will be discussed in the functions section. 

- No data. Here we will also need to find nearest neighbors and then use this to inform the function that is used. 

Constraints? :

- totals for foretak
- reg_type 03 / 04?
- ikke i drift?
- totals for salgsint / forbruk ? How many non varehandel bedrift?
- relationship between lønn, varekostnader, driftkostnader

# Example foretak:

##### Good data
Orgnr_f : 879263662
##### Bad Data (missing info on one or more units)
Orgnr_f : 934531345
##### No Data
Orgnr_f : 

# Basic function


Were basically talking about a constrained optimisation problem. We need to investigate several functions and determine which one is better. It may in fact be that some functions work better depending on the situtation - ideally an algorythm would determine which one is better - but for fun perhaps we can have a toggle option so users can look through which one they think works best. 

Options for functions might include: 

- using neighest neighbors and their averages in a function to calculate a bedrifters sales. After this 

Constrained optimization in the context of machine learning typically involves defining an objective function that you want to minimize or maximize while keeping certain constraints satisfied. In your case, the constraint is that the sum of all estimated revenues for 'bedrifter' under a single 'foretak' must equal the known total revenue for that 'foretak'.

Here's how you might proceed with a constrained optimization approach to predict the percentage distribution of revenues:

1. **Normalize the Data**: Transform the revenue of each 'bedrift' into a proportion of the total revenue of its parent 'foretak'. This is your target variable for modeling.

2. **Feature Selection**: Choose features that could influence the revenue distribution among 'bedrifter'. For instance, the number of employees, the industry sector, and historical sales data.

3. **Model Selection**: Select a suitable regression model. Given that you have a constraint that needs to be satisfied, you may want to look into models that can naturally handle constraints, like a linear regression model with non-negativity constraints or more advanced methods like quadratic programming.

4. **Loss Function**: Define a loss function for the model training. This is what the model will minimize. A common choice is Mean Squared Error (MSE) for regression problems.

5. **Constraint Handling**: To ensure that the predicted proportions for each 'foretak' sum to 1 (or 100%), you can incorporate a post-processing step that proportionally scales the predictions. Alternatively, you can build this constraint directly into the model training using a custom optimization algorithm.

6. **Model Training**: Train the model on your data, minimizing the loss function while ensuring that the constraints are satisfied.

7. **Model Evaluation**: Evaluate the model using a validation set or through cross-validation to ensure it generalizes well to unseen data.

8. **Prediction and Rescaling**: Use the model to predict the revenue proportions for all 'bedrifter'. Rescale the predictions so that they sum to 1 for each 'foretak'.

Here's an example using Python's `scipy.optimize` module, which allows for constrained optimization:

```python
from scipy.optimize import minimize
import numpy as np

# Assume X_train contains the features for your bedrifter
# and y_train contains the normalized revenue (as a percentage of the foretak's total revenue).

# Define the objective function (e.g., mean squared error)
def objective_function(coefs, X, y):
    predictions = X.dot(coefs)
    return ((predictions - y) ** 2).mean()

# Define the constraint function (sum of predictions must equal 1 for each foretak)
def constraint_function(coefs, X):
    return X.dot(coefs).sum() - 1

# The constraint in a form scipy can use
constraint = {'type': 'eq', 'fun': constraint_function, 'args': (X_train,)}

# Initial guess for the coefficients
initial_coefs = np.zeros(X_train.shape[1])

# Perform the optimization
result = minimize(
    fun=objective_function,
    x0=initial_coefs,
    args=(X_train, y_train),
    constraints=constraint,
    method='SLSQP',  # Sequential Least Squares Programming
    options={'disp': True}
)

# The optimal coefficients are in result.x
optimal_coefs = result.x

# Predict the revenue percentages
y_pred = X_train.dot(optimal_coefs)
```

In this example, `minimize` is used to find the coefficients of your features that minimize the mean squared error of your predictions, subject to the constraint that the predictions sum to 1. The `'SLSQP'` method supports both equality and inequality constraints.

This is a simplified example and assumes that all 'bedrifter' in your dataset belong to a single 'foretak'. If you have multiple 'foretak', you would need to modify your constraint function to ensure that the sum of the predictions for each group of 'bedrifter' belonging to the same 'foretak' equals the total revenue of that 'foretak'.

In practice, you may have to employ more sophisticated techniques to handle multiple constraints (one for each 'foretak'). You might need to run the optimization separately for each 'foretak' or use more complex optimization methods that can handle group-wise constraints.

# Choosing the right function:

We can build several ourselves and test them directly. Or we can use machine learning to detmerine what the correct one is. 

## Our own functions

Possible contraints:

- contrained by foretak totals
- NO-poster - can we use

## Machine learning function

If the optimal function varies by industry and the goal is to minimize error across different segments such as 'bedrifter' under various 'foretak' and industries, then one approach is to use a machine learning model that can learn the function from the data. This model would be trained to minimize the prediction error and could adapt to different industries by learning from industry-specific patterns in the training data.

Here are some steps to build such a model:

### Step 1: Feature Engineering
Include industry as a categorical feature in your dataset. If you have other features that vary by industry and could influence revenue distribution, include those as well.

### Step 2: Model Selection
Choose a flexible machine learning model that can capture complex relationships in the data. Models like Random Forests, Gradient Boosting Machines (like XGBoost, LightGBM), or Neural Networks can automatically learn to adjust their predictions based on the input features, which would include the industry category.

### Step 3: Model Training
Train your model on the training data, using a loss function that represents the error you want to minimize (typically Mean Squared Error for regression tasks). The model should learn to make different predictions for 'bedrifter' in different industries, as it will use the industry feature to adjust its internal decision rules.

### Step 4: Cross-Validation
Use cross-validation to evaluate the model's performance across different subsets of your data. This helps ensure that your model not only minimizes error on the training data but also generalizes well to new, unseen data.

### Step 5: Constrained Optimization Post-Processing
After you've trained your model and made predictions, you might find that the predicted revenues for 'bedrifter' under a single 'foretak' don't sum up to the total known revenue for that 'foretak'. You can then apply a constrained optimization or scaling post-processing step to adjust the predictions so that they meet this constraint.

For instance, if the predicted revenues for the 'bedrifter' under one 'foretak' sum up to 90% of the known total revenue for that 'foretak', you would scale up each 'bedrift's predicted revenue by the same factor so that they sum to the total.

### Step 6: Implementation
Here's a very high-level pseudocode for implementing the above strategy using a gradient boosting machine like XGBoost:

```python
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

# Prepare your data
# X = features including industry, location, etc.
# y = normalized revenue targets (proportions of total 'foretak' revenue)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Rescale predictions so that they sum to the total 'foretak' revenue
# This would be done for each 'foretak' group within your dataset
# For simplicity, let's assume 'total_revenue_by_foretak' is a dictionary
# mapping each 'foretak' to its total revenue
for foretak, total_revenue in total_revenue_by_foretak.items():
    # Select the predictions for the current 'foretak'
    pred_indices = X_test['foretak'] == foretak
    foretak_preds = y_pred[pred_indices]
    
    # Calculate the scaling factor
    scaling_factor = total_revenue / sum(foretak_preds)
    
    # Scale the predictions
    y_pred[pred_indices] *= scaling_factor

# Calculate the final mean squared error after rescaling
final_mse = mean_squared_error(y_test, y_pred)
print(f"Final Mean Squared Error: {final_mse}")
```

This pseudocode outlines the training and prediction steps and includes a simple rescaling procedure to satisfy the revenue constraints. The key is to have 'foretak' as an identifier in your features so that you can group predictions accordingly. 

The actual implementation will be more complex, especially the rescaling part, as it needs to handle grouping and apply the scaling correctly for each group. The exact details would depend on your dataset's structure and the business logic that determines how 'bedrifter' are grouped under 'foretak'.

# Get user input

In [ ]:
# Ask the user for the year
# 879263662

year = input("Please enter the year: ")

enhet = input("Please enter the orgnr_foretak #: ")

# Read in geographical data

### Current year and month

In [ ]:
import datetime

# Get the current date
current_date = datetime.datetime.now()

# Format the year and month
current_year = current_date.strftime("%Y")
current_month = current_date.strftime("%m")

# Subtract one day from the first day of the current month to get the last day of the previous month
last_day_of_previous_month = datetime.datetime(
    current_date.year, current_date.month, 1
) - datetime.timedelta(days=1)

# Now we can get the month number of the previous month
previous_month = last_day_of_previous_month.strftime("%m")

VOFSTI = (
    "ssb-prod-vof-data-delt/stedfesting-situasjonsuttak_data/klargjorte-data/parquet"
)
file_path = (
    f"{VOFSTI}/stedfesting-situasjonsuttak_p{current_year}-{previous_month}_v1.parquet"
)

print(file_path)

In [ ]:
vof_df = dp.read_pandas(f"{file_path}")
vof_gdf = gpd.GeoDataFrame(
    vof_df,
    geometry=gpd.points_from_xy(
        vof_df["y_koordinat"],
        vof_df["x_koordinat"],
    ),
    crs=25833,
)

vof_gdf = vof_gdf.rename(
    columns={
        "orgnrbed": "orgnr_bedrift",
        "org_nr": "orgnr_foretak",
        "nace1_sn07": "naring",
    }
)


vof_gdf = vof_gdf[
    [
        "orgnr_bedrift",
        "orgnr_foretak",
        "naring",
        "x_koordinat",
        "y_koordinat",
        "rute_100m",
        "rute_1000m",
        "geometry",
    ]
]
pd.set_option("display.max_columns", None)

vof_gdf = vof_gdf.dropna(subset=["x_koordinat"])
vof_gdf = vof_gdf.drop_duplicates(subset="orgnr_bedrift")
vof_gdf = vof_gdf.drop("orgnr_foretak", axis=1)
vof_gdf.head()

# Get bedrift data

In [ ]:
fil_path = f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet"

bedrifter = pd.read_parquet(fil_path, filesystem=fs)

# Create 'nace4' by slicing the first 5 characters of 'naring'
bedrifter["naring4"] = bedrifter["naring"].str[:5]

bedrifter["naring_f_4"] = bedrifter["naring_f"].str[:5]

# Create 'nace3' by slicing the first 4 characters of 'naring'
bedrifter["naring3"] = bedrifter["naring"].str[:4]

bedrifter["naring_f_3"] = bedrifter["naring_f"].str[:4]

enhets_id = bedrifter.loc[bedrifter["orgnr_foretak"] == enhet, "enhets_id"].values[0]


# Now 'bedrifter' has two new columns: 'nace4' and 'nace3'


# bedrifter.head()

# Assuming 'bedrifter' is your DataFrame, 'enhet' is your variable with the stored value

# Create the 'naring3' list for rows where 'orgnr_foretak' equals 'enhet'
naring3_list = list(
    bedrifter.loc[bedrifter["orgnr_foretak"] == enhet, "naring3"].unique()
)

# Create the 'naring_f_3' list for rows where 'orgnr_f' equals 'enhet'

naring_f_3_list = list(
    bedrifter.loc[bedrifter["orgnr_foretak"] == enhet, "naring_f_3"].unique()
)
# Now you have two lists: 'naring3_list' and 'naring_f_3_list'


# Assuming 'naring3_list' and 'naring_f_3_list' have been defined as per your previous instructions

# Filter the DataFrame
filtered_bedrifter = bedrifter[
    bedrifter["naring3"].isin(naring3_list)
    | bedrifter["naring_f_3"].isin(naring_f_3_list)
]

filtered_bedrifter = filtered_bedrifter[
    [
        "orgnr_bedrift",
        "orgnr_foretak",
        "omsetning",
        "kommune",
        "naring_f",
        "naring4",
        "naring_f_4",
        "naring3",
        "naring_f_3",
        "nopost_driftskostnader",
        "sysselsetting_syss",
        "navn",
        "reg_type",
    ]
]

# Calculate the counts for each 'orgnr_foretak' and create a new column 'bedrift_count'
filtered_bedrifter["bedrift_count"] = filtered_bedrifter.groupby("orgnr_foretak")[
    "orgnr_foretak"
].transform("count")

# Now 'filtered_bedrifter' contains only the rows that meet the conditions
filtered_bedrifter.head()

# Get foretak data

In [ ]:
fil_path = f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_pub.parquet"

foretak = pd.read_parquet(fil_path, filesystem=fs)

foretak = foretak[
    ["orgnr_foretak", "omsetning", "sysselsetting_syss", "nopost_driftskostnader"]
]

foretak = foretak.rename(
    columns={
        "omsetning": "omsetning_foretak",
        "sysselsetting_syss": "sysselsetting_foretak",
        "nopost_driftskostnader": "nopost_driftskostnader_foretak",
    }
)

# Need to do this in order to calculate other variables. Not the true syss count.
foretak["sysselsetting_foretak"].fillna(1, inplace=True)
foretak["sysselsetting_foretak"].replace(0, 1, inplace=True)  # Replace 0 with 1

# Merge foretak and bedrifter data

In [ ]:
filtered_bedrifter = filtered_bedrifter.merge(foretak, on="orgnr_foretak", how="left")
filtered_bedrifter = filtered_bedrifter.drop_duplicates(
    subset="orgnr_bedrift", keep="first"
)

# filtered_bedrifter = filtered_bedrifter.drop("orgnr_foretak", axis=1)

In [ ]:
filtered_bedrifter.head()

In [ ]:
# Ensure that there are no zero values in the denominators to avoid division by zero errors
filtered_bedrifter["sysselsetting_foretak"].replace(0, np.nan, inplace=True)
filtered_bedrifter["bedrift_count"].replace(0, np.nan, inplace=True)

# Create new columns with the calculated values
filtered_bedrifter["oms_per_syss_foretak"] = (
    filtered_bedrifter["omsetning_foretak"]
    / filtered_bedrifter["sysselsetting_foretak"]
)
filtered_bedrifter["oms_per_bedrift_foretak"] = (
    filtered_bedrifter["omsetning_foretak"] / filtered_bedrifter["bedrift_count"]
)

# Display the DataFrame to verify the results

In [ ]:
# filtered_bedrifter.head()

# Explore the map

In [ ]:
vof_gdf.head()

In [ ]:
merged_df = filtered_bedrifter.merge(vof_gdf, on="orgnr_bedrift", how="left")
merged_gdf = gpd.GeoDataFrame(merged_df, geometry="geometry")

In [ ]:
filtered_df = merged_gdf[merged_gdf["orgnr_foretak"] == enhet]
# filtered_df

In [ ]:
# filtered_df

In [ ]:
# merged_gdf = merged_gdf.dropna(subset=["x_koordinat", "y_koordinat"])
# sg.explore(merged_gdf)

# Prepare data for machine learning

In [ ]:
features = [
    "x_koordinat",
    "y_koordinat",
    "sysselsetting_syss",
    "naring",
    "omsetning_foretak",
    "nopost_driftskostnader_foretak",
    "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]

non_feature_columns = ["orgnr_foretak", "orgnr_bedrift"]  # Non-feature columns to keep

merged_gdf = merged_gdf.dropna(subset=features + non_feature_columns)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assume 'merged_gdf' is your DataFrame and it has been preprocessed to include the following columns:
# 'x_koordinat', 'y_koordinat', 'sysselsetting_syss', 'naring', 'omsetning'

# Preprocessing
# StandardScaler will scale the coordinates and number of employees
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            [
                "x_koordinat",
                "y_koordinat",
                "sysselsetting_syss",
                "omsetning_foretak",
                "nopost_driftskostnader_foretak",
                "oms_per_syss_foretak",
                "oms_per_bedrift_foretak",
            ],
        ),
        ("cat", OneHotEncoder(), ["naring"]),  # One-hot encode 'naring'
    ]
)

# Model setup
knn_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("knn", KNeighborsRegressor(n_neighbors=5)),  # Adjust n_neighbors as needed
    ]
)

# Fit the model
features = [
    "x_koordinat",
    "y_koordinat",
    "sysselsetting_syss",
    "naring",
    "omsetning_foretak",
    "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]
target = "omsetning"

knn_pipeline.fit(merged_gdf[features], merged_gdf[target])

# To predict, you would follow a similar approach to the previous examples,
# ensuring that you pass the x and y coordinates directly to the model after scaling.

# Train the data

In [ ]:
# from sklearn.model_selection import train_test_split, GroupShuffleSplit
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error, r2_score

# # Replace 'target_variable_name' with the actual name of your target variable
# features = [
#     "x_koordinat",
#     "y_koordinat",
#     # "sysselsetting_syss",
#     "naring",
#     "omsetning_foretak",
#     "nopost_driftskostnader_foretak",
#     "oms_per_syss_foretak",
#     "oms_per_bedrift_foretak",
# ]
# target = "omsetning"

# # Preprocessing
# # Define the preprocessing for numerical and categorical features
# preprocessor = ColumnTransformer(
#     transformers=[
#         (
#             "num",
#             StandardScaler(),
#             [
#                 "x_koordinat",
#                 "y_koordinat",
#                 # "sysselsetting_syss",
#                 "naring",
#                 "omsetning_foretak",
#                 "nopost_driftskostnader_foretak",
#                 "oms_per_syss_foretak",
#                 "oms_per_bedrift_foretak",
#             ],
#         ),
#         (
#             "cat",
#             OneHotEncoder(handle_unknown="ignore"),
#             ["naring"],
#         ),  # Set handle_unknown to 'ignore'
#     ]
# )


# # Model setup
# # Setup the pipeline with the preprocessing and the KNN model
# knn_pipeline = Pipeline(
#     steps=[
#         ("preprocessor", preprocessor),
#         ("knn", KNeighborsRegressor(n_neighbors=4)),  # Adjust n_neighbors as needed
#     ]
# )

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(
#     merged_gdf[features + ['orgnr_bedrift', 'reg_type', 'orgnr_foretak']],
#     merged_gdf[target],
#     test_size=0.2,  # 20% for testing
#     random_state=42,  # Seed for reproducibility
# )

# # Train the model
# knn_pipeline.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = knn_pipeline.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {mse}")

# # Calculate Mean Absolute Error (MAE)
# mae = mean_absolute_error(y_test, y_pred)

# # Calculate Root Mean Squared Error (RMSE)
# rmse = mean_squared_error(y_test, y_pred, squared=False)

# # Calculate R^2 Score
# r2 = r2_score(y_test, y_pred)

# mse, mae, rmse, r2


#########################################

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

# Replace 'target_variable_name' with the actual name of your target variable
features = [
    "x_koordinat",
    "y_koordinat",
    "sysselsetting_syss",
    "naring",
    "omsetning_foretak",
    "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]
target = "omsetning"

# Preprocessing
# Define the preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            [
                "x_koordinat",
                "y_koordinat",
                "sysselsetting_syss",
                "omsetning_foretak",
                "nopost_driftskostnader_foretak",
                "oms_per_syss_foretak",
                "oms_per_bedrift_foretak",
            ],
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            ["naring"],
        ),
    ]
)

# Model setup
# Setup the pipeline with the preprocessing and the KNN model
knn_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("knn", KNeighborsRegressor(n_neighbors=5)),
    ]
)

# GroupShuffleSplit
group_split = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in group_split.split(
    merged_gdf, groups=merged_gdf["orgnr_foretak"]
):
    X_train, X_test = merged_gdf.iloc[train_idx], merged_gdf.iloc[test_idx]
    y_train, y_test = (
        merged_gdf[target].iloc[train_idx],
        merged_gdf[target].iloc[test_idx],
    )

# Train the model
knn_pipeline.fit(X_train[features], y_train)

# Make predictions on the test set
y_pred = knn_pipeline.predict(X_test[features])

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R^2 Score: {r2}")

In [ ]:
import pandas as pd

# Create a DataFrame to store the results
adjusted_result_df = pd.DataFrame(
    X_test, columns=features + ["orgnr_bedrift", "reg_type", "orgnr_foretak"]
)
adjusted_result_df["actual_omsetning"] = y_test

# Add the predicted 'omsetning' from y_pred
adjusted_result_df["predicted_omsetning"] = y_pred

# adjusted_result_df["diff"] = (adjusted_result_df["predicted_omsetning"] - adjusted_result_df["actual_omsetning"]).abs()
# # adjusted_result_df.sort_values(by='diff', inplace=True)

In [ ]:
# adjusted_result_df.head(50)

In [ ]:
# Group by 'orgnr_foretak' and sum 'predicted_omsetning'
total_predicted = (
    adjusted_result_df.groupby("orgnr_foretak")["predicted_omsetning"]
    .sum()
    .reset_index()
)
total_predicted.rename(
    columns={"predicted_omsetning": "total_predicted_for_foretak"}, inplace=True
)

In [ ]:
adjusted_result_df = adjusted_result_df.merge(
    total_predicted, on="orgnr_foretak", how="left"
)

In [ ]:
adjusted_result_df.head()

In [ ]:
# Now 'total_predicted' is a DataFrame with 'orgnr_foretak' and the corresponding 'total_predicted_for_foretak'

# Calculate the percentage share of each 'orgnr_bedrift' prediction relative to its 'orgnr_foretak'
adjusted_result_df["predicted_share"] = (
    adjusted_result_df["predicted_omsetning"]
    / adjusted_result_df["total_predicted_for_foretak"]
)

In [ ]:
adjusted_result_df.head()

In [ ]:
# Calculate the new adjusted predicted 'omsetning' based on the percentage share
adjusted_result_df["new_predicted"] = (
    adjusted_result_df["predicted_share"] * adjusted_result_df["omsetning_foretak"]
)
pd.options.display.float_format = "{:.2f}".format
# Now adjusted_result_df contains the adjusted predictions displayed as whole numbers
# adjusted_result_df.head(50)

In [ ]:
# Filter the DataFrame to include only rows where reg_type is '02'
reg_type_02_df = adjusted_result_df[adjusted_result_df["reg_type"] == "02"]

In [ ]:
reg_type_02_df["diff"] = (
    reg_type_02_df["actual_omsetning"] - reg_type_02_df["new_predicted"]
).abs()

# Continue with your analysis using the filtered DataFrame

In [ ]:
# Group by 'orgnr_foretak' and sum 'diff' for each group
sum_diff_per_orgnr = reg_type_02_df.groupby("orgnr_foretak")["diff"].sum().reset_index()

# Rename the summed 'diff' column for clarity
sum_diff_per_orgnr = sum_diff_per_orgnr.rename(columns={"diff": "sum_diff"})

# Merge this sum back into the original DataFrame
reg_type_02_df_merged = reg_type_02_df.merge(sum_diff_per_orgnr, on="orgnr_foretak")

# Sort the merged DataFrame by 'sum_diff'
sorted_reg_type_02_df = reg_type_02_df_merged.sort_values(by="sum_diff")


# Set the maximum number of rows to display
pd.set_option("display.max_rows", None)  # This will display all rows

# Display the sorted DataFrame
# sorted_reg_type_02_df.head(500)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate Mean Absolute Error (MAE) using the adjusted predictions
mae_new_predicted = mean_absolute_error(
    sorted_df["actual_omsetning"], sorted_df["new_predicted"]
)
r2_new_predicted = r2_score(sorted_df["actual_omsetning"], sorted_df["new_predicted"])
print(
    f"Mean Absolute Error (MAE) for reg_type using new_predicted: {mae_new_predicted}"
)
print(f"R-squared (R2) for reg_type 2 using new_predicted: {r2_new_predicted}")

In [ ]:
adjusted_result_df = adjusted_result_df.fillna(0)

In [ ]:
mae_new_predicted = mean_absolute_error(
    adjusted_result_df["actual_omsetning"], adjusted_result_df["new_predicted"]
)
r2_new_predicted = r2_score(
    adjusted_result_df["actual_omsetning"], adjusted_result_df["new_predicted"]
)
print(
    f"Mean Absolute Error (MAE) for the whole naring new_predicted: {mae_new_predicted}"
)
print(f"R-squared (R2) for the whole naring using new_predicted: {r2_new_predicted}")

# Get results for entire dataframe

In [ ]:
# # Create a DataFrame with actual and predicted values
# results_df = pd.DataFrame({
#     'Actual': y_test,
#     'Predicted': y_pred
# })

# # Reset the index of the DataFrame if necessary (to ensure alignment)
# results_df = results_df.reset_index(drop=True)

# # Display the results
# results_df.head(40)

# Convert the predictions to a DataFrame
predicted_df = pd.DataFrame(y_pred, columns=["Predicted"])

# Reset the index on the test set to align with the predicted DataFrame
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Convert the actual and predicted values to a DataFrame
results_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})

# Combine the test set with the actual and predicted values
results_df = pd.concat([X_test, results_df], axis=1)

# Calculate the difference between the actual and predicted values
results_df["Difference"] = results_df["Actual"] - results_df["Predicted"]

# Display the results
results_df.head(10)

In [ ]:
# Filter the DataFrame for rows where 'reg_type' is '02'
filtered_df = results_df[results_df["reg_type"] == "02"]

# Extract the actual and predicted values for the filtered rows
filtered_actual = filtered_df["Actual"]
filtered_predicted = filtered_df["Predicted"]

# Evaluate the model on the filtered data
mse_filtered = mean_squared_error(filtered_actual, filtered_predicted)
print(f"Mean Squared Error for 'reg_type' 02: {mse_filtered}")

# Calculate Mean Absolute Error (MAE) for the filtered data
mae_filtered = mean_absolute_error(filtered_actual, filtered_predicted)

# Calculate Root Mean Squared Error (RMSE) for the filtered data
rmse_filtered = mean_squared_error(filtered_actual, filtered_predicted, squared=False)

# Calculate R^2 Score for the filtered data
r2_filtered = r2_score(filtered_actual, filtered_predicted)

# Print the evaluation metrics for the filtered data
print(f"Mean Absolute Error for 'reg_type' 02: {mae_filtered}")
print(f"Root Mean Squared Error for 'reg_type' 02: {rmse_filtered}")
print(f"R^2 Score for 'reg_type' 02: {r2_filtered}")

# Get results just for chosen foretak

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Define your features as before
features = [
    "x_koordinat",
    "y_koordinat",
    "sysselsetting_syss",
    "naring",
    "omsetning_foretak",
    "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]

# Reset the index of the DataFrame to make sure it's in sequential order after dropping NaNs
merged_gdf.reset_index(drop=True, inplace=True)

# Fit the NearestNeighbors model using only the features
nn = NearestNeighbors()
nn.fit(merged_gdf[features])

# Find the instances of the specific 'orgnr_foretak' in the dataset
orgnr_foretak_value = (
    f"{enhet}"  # Replace with the actual 'orgnr_foretak' you're interested in
)
specific_orgnr_indices = merged_gdf.index[
    merged_gdf["orgnr_foretak"] == orgnr_foretak_value
].tolist()

# For each instance of 'orgnr_foretak', find its nearest neighbors
all_neighbors_indices = []
for index in specific_orgnr_indices:
    distances, indices = nn.kneighbors(
        [merged_gdf.loc[index, features].values], n_neighbors=5
    )
    all_neighbors_indices.extend(indices.flatten())

# Remove duplicates and the indices of the 'orgnr_foretak' itself
all_neighbors_indices = list(set(all_neighbors_indices) - set(specific_orgnr_indices))

# Create a DataFrame of the nearest neighbors
neighbors_df = merged_gdf.iloc[all_neighbors_indices]

# Combine the specific 'orgnr_foretak' rows with the neighbors
final_df = pd.concat([merged_gdf.iloc[specific_orgnr_indices], neighbors_df])

final_df = final_df.drop_duplicates(subset="orgnr_bedrift")


# The resulting 'final_df' will have all the same columns as 'merged_gdf' but only contain the rows for the chosen 'orgnr_foretak' and its neighbors.

In [ ]:
# sg.explore(nærmeste_naboer, "omsetning")
# final_df.head(100)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Define your features as before
features = [
    "x_koordinat",
    "y_koordinat",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]

# Reset the index of the DataFrame to make sure it's in sequential order after dropping NaNs
merged_gdf.reset_index(drop=True, inplace=True)

# Fit the NearestNeighbors model using only the features
nn = NearestNeighbors()
nn.fit(merged_gdf[features])

# Find the instances of the specific 'orgnr_foretak' in the dataset
orgnr_foretak_value = (
    f"{enhet}"  # Replace with the actual 'orgnr_foretak' you're interested in
)
specific_orgnr_indices = merged_gdf.index[
    merged_gdf["orgnr_foretak"] == orgnr_foretak_value
].tolist()

# For each instance of 'orgnr_foretak', find its nearest neighbors
all_neighbors_indices = []
orgnr_bedrift_neighbors = (
    {}
)  # This dictionary will map each 'orgnr_bedrift' to its neighbors
for index in specific_orgnr_indices:
    distances, indices = nn.kneighbors(
        [merged_gdf.loc[index, features].values], n_neighbors=5
    )
    all_neighbors_indices.extend(indices.flatten())
    orgnr_bedrift_neighbors[
        merged_gdf.loc[index, "orgnr_bedrift"]
    ] = indices.flatten()  # Add the neighbors to the dictionary

# Remove duplicates and the indices of the 'orgnr_foretak' itself
all_neighbors_indices = list(set(all_neighbors_indices) - set(specific_orgnr_indices))

# Create a DataFrame of the nearest neighbors
neighbors_df = merged_gdf.iloc[all_neighbors_indices]

# Add a new column to 'neighbors_df' that indicates which 'orgnr_bedrift' each neighbor belongs to
neighbors_df["orgnr_bedrift_neighbor"] = neighbors_df.index.map(
    lambda x: [k for k, v in orgnr_bedrift_neighbors.items() if x in v]
)

# Combine the specific 'orgnr_foretak' rows with the neighbors
nærmeste_naboer = pd.concat([merged_gdf.iloc[specific_orgnr_indices], neighbors_df])

In [ ]:
# units of a foretak cant be neighbors to each other:

# Filter out the instances of the specific 'orgnr_foretak'
orgnr_foretak_value = f"{enhet}"  # The 'orgnr_foretak' you're interested in
filtered_gdf = merged_gdf[merged_gdf["orgnr_foretak"] != orgnr_foretak_value]

# Fit the NearestNeighbors model on the filtered DataFrame
nn = NearestNeighbors()
nn.fit(filtered_gdf[features])

# Find the instances of the specific 'orgnr_foretak' in the original DataFrame
specific_orgnr_indices = merged_gdf.index[
    merged_gdf["orgnr_foretak"] == orgnr_foretak_value
].tolist()

# For each instance of 'orgnr_foretak', find its nearest neighbors in the filtered DataFrame
all_neighbors_indices = []
for index in specific_orgnr_indices:
    distances, indices = nn.kneighbors(
        [merged_gdf.loc[index, features].values], n_neighbors=5
    )
    all_neighbors_indices.extend(indices.flatten())

# Remove duplicates from the neighbors' indices
all_neighbors_indices = list(set(all_neighbors_indices))

# Create a DataFrame of the nearest neighbors
neighbors_df = filtered_gdf.iloc[all_neighbors_indices]

# Combine the specific 'orgnr_foretak' rows with the neighbors
nærmeste_naboer = pd.concat([merged_gdf.iloc[specific_orgnr_indices], neighbors_df])

In [ ]:
# final_df.head(200)

sg.explore(nærmeste_naboer, "orgnr_foretak")

# nærmeste_naboer

In [ ]:
sg.explore(final_df)

# Train the model to find the best parameters

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    make_scorer,
)

# Assume 'merged_gdf' is your DataFrame and it has been preprocessed to include the following columns:
# 'x_koordinat', 'y_koordinat', 'sysselsetting_syss', 'naring', 'omsetning'

# Define your features and target variable
features = [
    "x_koordinat",
    "y_koordinat",
    "sysselsetting_syss",
    "naring",
    "omsetning_foretak",
    "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]
target = "omsetning"

# Preprocessing
# Define the preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            [
                "x_koordinat",
                "y_koordinat",
                "sysselsetting_syss",
                "naring",
                "omsetning_foretak",
                "nopost_driftskostnader_foretak",
                "oms_per_syss_foretak",
                "oms_per_bedrift_foretak",
            ],
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            ["naring"],
        ),
    ]
)

# Model setup
# Setup the pipeline with the preprocessing and the KNN model
knn_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("knn", KNeighborsRegressor()),
    ]
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    merged_gdf[features + ["orgnr_bedrift", "reg_type", "orgnr_foretak"]],
    merged_gdf[target],
    test_size=0.2,  # 20% for testing
    random_state=42,  # Seed for reproducibility
)

# Define the parameter grid to search over
# param_grid = {
#     'knn__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'knn__weights': ['uniform', 'distance'],
#     # Add other parameters here if you wish
# }

param_grid = {
    "knn__n_neighbors": [2, 3, 4, 5],
    "knn__weights": ["uniform", "distance"],
    # 'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    # 'knn__leaf_size': [10, 20, 30, 40, 50],
    "knn__p": [1, 2],
    "knn__metric": ["euclidean", "manhattan", "minkowski"],
}


def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return adjusted_r2


# Create a scorer
adjusted_r2_scorer = make_scorer(
    adjusted_r2_score, greater_is_better=True, n=X_train.shape[0], p=X_train.shape[1]
)

# Create a GridSearchCV object
grid_search = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring=adjusted_r2_scorer)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print(f"Best parameters: {grid_search.best_params_}")

# Use the best estimator to make predictions on the test set
y_pred = grid_search.predict(X_test)


predicted_df = pd.DataFrame(y_pred, columns=["Predicted"])

# Reset the index on the test set to align with the predicted DataFrame
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Convert the actual and predicted values to a DataFrame
results_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})

# Combine the test set with the actual and predicted values
results_df = pd.concat([X_test, results_df], axis=1)

# Calculate the difference between the actual and predicted values
results_df["Difference"] = results_df["Actual"] - results_df["Predicted"]

# Filter the DataFrame for rows where 'reg_type' is '02'
filtered_df = results_df[results_df["reg_type"] == "02"]

# Extract the actual and predicted values for the filtered rows
filtered_actual = filtered_df["Actual"]
filtered_predicted = filtered_df["Predicted"]

# Evaluate the model on the filtered data
mse_filtered = mean_squared_error(filtered_actual, filtered_predicted)
print(f"Mean Squared Error for 'reg_type' 02: {mse_filtered}")

# Calculate Mean Absolute Error (MAE) for the filtered data
mae_filtered = mean_absolute_error(filtered_actual, filtered_predicted)

# Calculate Root Mean Squared Error (RMSE) for the filtered data
rmse_filtered = mean_squared_error(filtered_actual, filtered_predicted, squared=False)

# Calculate R^2 Score for the filtered data
r2_filtered = r2_score(filtered_actual, filtered_predicted)

# Number of observations
n = X_test.shape[0]

# Number of features (predictors)
p = X_test.shape[1]

# Calculate Adjusted R^2 Score
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Print the evaluation metrics for the filtered data
print(f"Mean Absolute Error for 'reg_type' 02: {mae_filtered}")
print(f"Root Mean Squared Error for 'reg_type' 02: {rmse_filtered}")
print(f"R^2 Score for 'reg_type' 02: {r2_filtered}")
print(f"Adjusted R^2 Score: {adjusted_r2}")

With Everything:
    
Best parameters: {'knn__n_neighbors': 4, 'knn__weights': 'uniform'}
Mean Squared Error: 6194897820.853139
Mean Absolute Error: 13858.122807017544
Root Mean Squared Error: 78707.67320187492
R^2 Score: 0.8251536639477731
Adjusted R^2 Score: 0.824382991392171

Without driftskostnader:

Best parameters: {'knn__n_neighbors': 5, 'knn__weights': 'uniform'}
Mean Squared Error: 8963619983.572657
Mean Absolute Error: 15219.74111842105
Root Mean Squared Error: 94676.39612687344
R^2 Score: 0.747008561365365

Without geopgraphical data:

Best parameters: {'knn__n_neighbors': 5, 'knn__weights': 'uniform'}
Mean Squared Error: 8692302957.268972
Mean Absolute Error: 12964.328618421052
Root Mean Squared Error: 93232.52092091565
R^2 Score: 0.7546662805610065

Without per foretak data and without driftskostnader:

Best parameters: {'knn__n_neighbors': 10, 'knn__weights': 'uniform'}
Mean Squared Error: 22358440708.48245
Mean Absolute Error: 22249.31398026316
Root Mean Squared Error: 149527.3911645704
R^2 Score: 0.3689498114787729

Without syss and without driftskostnader:

Best parameters: {'knn__n_neighbors': 7, 'knn__weights': 'uniform'}
Mean Squared Error: 3511909656.662437
Mean Absolute Error: 16206.316885964912
Root Mean Squared Error: 59261.367320223355
R^2 Score: 0.9008789888435484


Without syss:

Best parameters: {'knn__n_neighbors': 4, 'knn__weights': 'uniform'}
Mean Squared Error: 3208207771.3834634
Mean Absolute Error: 15201.000959429824
Root Mean Squared Error: 56641.04316997934
R^2 Score: 0.9094507463492871

Without syss or geographical location:

Best parameters: {'knn__n_neighbors': 3, 'knn__weights': 'uniform'}
Mean Squared Error: 3714965103.26931
Mean Absolute Error: 12404.040387426901
Root Mean Squared Error: 60950.51356034098
R^2 Score: 0.8951479014420503
Adjusted R^2 Score: 0.8948595293338051

# Train data only for reg_type 2

In [ ]:
filtered_df = merged_gdf[merged_gdf["reg_type"] == "02"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

# Replace 'target_variable_name' with the actual name of your target variable
features = [
    "x_koordinat",
    "y_koordinat",
    # "sysselsetting_syss",
    "naring",
    # "omsetning_foretak",
    # "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    # "oms_per_bedrift_foretak",
]
target = "omsetning"

# Preprocessing
# Define the preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            [
                "x_koordinat",
                "y_koordinat",
                # "sysselsetting_syss",
                "naring",
                # "omsetning_foretak",
                # "nopost_driftskostnader_foretak",
                "oms_per_syss_foretak",
                # "oms_per_bedrift_foretak",
            ],
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            ["naring"],
        ),  # Set handle_unknown to 'ignore'
    ]
)


# Model setup
# Setup the pipeline with the preprocessing and the KNN model
knn_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("knn", KNeighborsRegressor(n_neighbors=4)),  # Adjust n_neighbors as needed
    ]
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    filtered_df[features + ["orgnr_bedrift", "reg_type", "orgnr_foretak"]],
    filtered_df[target],
    test_size=0.2,  # 20% for testing
    random_state=42,  # Seed for reproducibility
)

# Train the model
knn_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calculate R^2 Score
r2 = r2_score(y_test, y_pred)

mse, mae, rmse, r2

In [ ]:
# # Create a DataFrame with actual and predicted values
# results_df = pd.DataFrame({
#     'Actual': y_test,
#     'Predicted': y_pred
# })

# # Reset the index of the DataFrame if necessary (to ensure alignment)
# results_df = results_df.reset_index(drop=True)

# # Display the results
# results_df.head(40)

# Convert the predictions to a DataFrame
predicted_df = pd.DataFrame(y_pred, columns=["Predicted"])

# Reset the index on the test set to align with the predicted DataFrame
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Convert the actual and predicted values to a DataFrame
results_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})

# Combine the test set with the actual and predicted values
results_df = pd.concat([X_test, results_df], axis=1)

# Calculate the difference between the actual and predicted values
results_df["Difference"] = results_df["Actual"] - results_df["Predicted"]

# Display the results


# Filter the DataFrame for rows where 'reg_type' is '02'
filtered_df = results_df[results_df["reg_type"] == "02"]

# Extract the actual and predicted values for the filtered rows
filtered_actual = filtered_df["Actual"]
filtered_predicted = filtered_df["Predicted"]

# Evaluate the model on the filtered data
mse_filtered = mean_squared_error(filtered_actual, filtered_predicted)
print(f"Mean Squared Error for 'reg_type' 02: {mse_filtered}")

# Calculate Mean Absolute Error (MAE) for the filtered data
mae_filtered = mean_absolute_error(filtered_actual, filtered_predicted)

# Calculate Root Mean Squared Error (RMSE) for the filtered data
rmse_filtered = mean_squared_error(filtered_actual, filtered_predicted, squared=False)

# Calculate R^2 Score for the filtered data
r2_filtered = r2_score(filtered_actual, filtered_predicted)

# Print the evaluation metrics for the filtered data
print(f"Mean Absolute Error for 'reg_type' 02: {mae_filtered}")
print(f"Root Mean Squared Error for 'reg_type' 02: {rmse_filtered}")
print(f"R^2 Score for 'reg_type' 02: {r2_filtered}")

In [ ]:
from sklearn.metrics import make_scorer


# Define a function to calculate adjusted R^2
def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return adjusted_r2


# Create a scorer
adjusted_r2_scorer = make_scorer(
    adjusted_r2_score, greater_is_better=True, n=X_train.shape[0], p=X_train.shape[1]
)

# Create a GridSearchCV object
grid_search = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring=adjusted_r2_scorer)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print(f"Best parameters: {grid_search.best_params_}")

# Linear Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming 'merged_gdf' is your DataFrame and it has the same columns as before

# Features and target variable
features = [
    "x_koordinat",
    "y_koordinat",
    "sysselsetting_syss",
    "naring",
    "omsetning_foretak",
    "nopost_driftskostnader_foretak",
    "oms_per_syss_foretak",
    "oms_per_bedrift_foretak",
]
target = "omsetning"

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            [
                "x_koordinat",
                "y_koordinat",
                "sysselsetting_syss",
                "omsetning_foretak",
                "nopost_driftskostnader_foretak",
                "oms_per_syss_foretak",
                "oms_per_bedrift_foretak",
            ],
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            ["naring"],
        ),
    ]
)

# Model setup
linear_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("linear_reg", LinearRegression()),
    ]
)

# GroupShuffleSplit
group_split = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in group_split.split(
    merged_gdf, groups=merged_gdf["orgnr_foretak"]
):
    X_train, X_test = merged_gdf.iloc[train_idx], merged_gdf.iloc[test_idx]
    y_train, y_test = (
        merged_gdf[target].iloc[train_idx],
        merged_gdf[target].iloc[test_idx],
    )

# Train the model
linear_pipeline.fit(X_train[features], y_train)

# Make predictions on the test set
y_pred = linear_pipeline.predict(X_test[features])

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R^2 Score: {r2}")

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# Assume knn_pipeline and linear_pipeline are your trained KNN and linear models

# Create a stacking regressor
stacked_model = StackingRegressor(
    estimators=[("knn", knn_pipeline), ("linear", linear_pipeline)],
    final_estimator=Ridge(),
)

# Train the stacked model
stacked_model.fit(X_train[features], y_train)


# Make predictions on the test set using the stacked model
y_pred_stacked = stacked_model.predict(X_test[features])

# Evaluate the stacked model
stacked_mse = mean_squared_error(y_test, y_pred_stacked)
stacked_r2 = r2_score(y_test, y_pred_stacked)
mae = mean_absolute_error(y_test, y_pred)

print(f"Stacked Model - Mean Squared Error: {stacked_mse}")
print(f"Stacked Model - R^2 Score: {stacked_r2}")
print(f"Mean Absolute Error: {mae}")